<a href="https://colab.research.google.com/github/naumanmirza1999/ASSIGNMENT6-KSBL/blob/main/NAUMAN_MIRZA_KSBL_ASSIGNMENT6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTING LIBRARIES
import pandas as pd
import numpy as np

In [2]:
# LOADING DATASET
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('DATASET.csv')

In [4]:
df.head()

,ID,TEXT,SENTIMENT
0,0,text,0
1,1,rising cases of covid does not alarm me rising...,1
2,2,please vote for chicagoindiaresolution marking...,0
3,3,wishing all of you eidaladha hazrat ibrahim as...,1
4,4,daily coronavirus cases in india top for first...,1


In [5]:
df = df.drop(0)

In [6]:
df.head()

,ID,TEXT,SENTIMENT
1,1,rising cases of covid does not alarm me rising...,1
2,2,please vote for chicagoindiaresolution marking...,0
3,3,wishing all of you eidaladha hazrat ibrahim as...,1
4,4,daily coronavirus cases in india top for first...,1
5,5,sitting here india style watching the raindrop...,0


In [9]:
df = df.drop(columns=['ID'])
df = df.reset_index(drop=True)

In [10]:
df.head()

,TEXT,SENTIMENT
0,rising cases of covid does not alarm me rising...,1
1,please vote for chicagoindiaresolution marking...,0
2,wishing all of you eidaladha hazrat ibrahim as...,1
3,daily coronavirus cases in india top for first...,1
4,sitting here india style watching the raindrop...,0
